In [ ]:
from tensorflow.keras.utils import to_categorical
from keras.datasets import mnist
from matplotlib import pyplot
from sklearn.model_selection import KFold
from keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Activation, Flatten
from keras.optimizers import SGD
from statistics import *
from keras.layers import BatchNormalization

In [ ]:
def load_dataset():

	(trainX, trainY), (testX, testY) = mnist.load_data()

	trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
	testX = testX.reshape((testX.shape[0], 28, 28, 1))

	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

In [ ]:
def prep_pixels(train, test):
	
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	
	return train_norm, test_norm

In [ ]:
def define_model():
  model = Sequential()
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
  model.add(BatchNormalization())
  model.add(MaxPooling2D((2, 2)))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
  model.add(MaxPooling2D((2, 2)))
  model.add(Flatten())
  model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
  model.add(BatchNormalization())
  model.add(Dense(10, activation='softmax'))

  opt = SGD(lr=0.01, momentum=0.9)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

In [ ]:
  def evaluate_model(dataX, dataY, n_folds=5):
	scores, histories = list(), list()
	
	kfold = KFold(n_folds, shuffle=True, random_state=1)
	
	for train_ix, test_ix in kfold.split(dataX):
		
		model = define_model()
		
		trainX, trainY, testX, testY = dataX[train_ix], dataY[train_ix], dataX[test_ix], dataY[test_ix]
		
		history = model.fit(trainX, trainY, epochs=10, batch_size=32, validation_data=(testX, testY), verbose=0)
		
		_, acc = model.evaluate(testX, testY, verbose=0)
		print('> %.3f' % (acc * 100.0))
		
		scores.append(acc)
		histories.append(history)
	return scores, histories

In [ ]:
def summarize_diagnostics(histories):
	for i in range(len(histories)):
	
		pyplot.subplot(2, 1, 1)
		pyplot.title('Cross Entropy Loss')
		pyplot.plot(histories[i].history['loss'], color='blue', label='train')
		pyplot.plot(histories[i].history['val_loss'], color='orange', label='test')
	
		pyplot.subplot(2, 1, 2)
		pyplot.title('Classification Accuracy')
		pyplot.plot(histories[i].history['accuracy'], color='blue', label='train')
		pyplot.plot(histories[i].history['val_accuracy'], color='orange', label='test')
	pyplot.show()

In [ ]:
def summarize_performance(scores):

	print('Accuracy: mean=%.3f std=%.3f, n=%d' % (mean(scores)*100, std(scores)*100, len(scores)))

	pyplot.boxplot(scores)
	pyplot.show()

In [ ]:
def run_test_harness():
	
	trainX, trainY, testX, testY = load_dataset()
	
	trainX, testX = prep_pixels(trainX, testX)
	
	scores, histories = evaluate_model(trainX, trainY)

	summarize_diagnostics(histories)

	summarize_performance(scores)
 
	model.fit(trainX, trainY, epochs=10, batch_size=32, verbose=0)

	model.save('final_model.h5')

In [ ]:
run_test_harness()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
def test_model():

	trainX, trainY, testX, testY = load_dataset()

	trainX, testX = prep_pixels(trainX, testX)

	model = load_model('final_model.h5')

	_, acc = model.evaluate(testX, testY, verbose=0)
	print('> %.3f' % (acc * 100.0))

In [ ]:
test_model()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

def load_image(filename):

	img = load_img(filename, grayscale=True, target_size=(28, 28))

	img = img_to_array(img)

	img = img.reshape(1, 28, 28, 1)

	img = img.astype('float32')
	img = img / 255.0
	return img

In [ ]:
def run_example():

	img = load_image("/content/drive/MyDrive/MIT FutureMakers/Day 11/sample_image.png")

	model = load_model('final_model.h5')

	digit = model.predict_classes(img)
	print(digit[0])

In [ ]:
run_example()